# Drinkwaterwinning Bleijerheide



# Analyse reeks Maastricht

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import pyextremes
from pathlib import Path

## Instellingen

In [ ]:
drempel = 0.1 # mm - lager dan dit is 'droog'
prec_hours = 24. # na een droogte tellen we de neerslag op over de eerste 24 uur
min_duration = 3*24 # we nemen alleen droogtes mee die langer duren dan 7 dagen

### Inlezen data

Inlezen van data: als True worden de ruwe KNMI files inglezen, als False een tijdelijke (snellere) file met alleen de uur-reeks.

In [ ]:
alldata  = pd.read_csv(r'brondata/alle_data.csv')
alldata.index = pd.to_datetime(alldata.iloc[:,0])

Zo ziet de reeks eruit tussen 1957 en 2023:

In [ ]:
Path('figuren_statistiek').mkdir(exist_ok=True, parents=False)
Path('tabellen_statistiek').mkdir(exist_ok=True, parents=False)
ax = alldata.plot()
ax.set_xlabel('')
ax.set_ylabel('Neerslag [mm/u]')
ax.set_title('Uurlijkse neerslag bij station Maastricht')
plt.savefig(r'figuren_statistiek/helereeks_uurlijks.png')

In [ ]:
alldata['YEAR'] = alldata.index.year
jaarsommen = alldata.groupby('YEAR').sum()['P']
ax = plt.subplot()
jaarsommen.plot()
ax.set_ylabel('Jaarsom [mm]')
ax.set_title('Jaarlijkse neerslag Maastricht')
ax.grid()
plt.savefig(r'figuren_statistiek/helereeks_jaarlijks.png')

In [ ]:
alldata.to_csv(r'tabellen_statistiek/alle_neerslagdata.csv',index=False, sep=",")

### Verzamel statistieken

In [ ]:
duration = []
prec_after = []
years = []
endd = []
n_event=0
duration.append(0)
prec_after.append(0.)
years.append(0)
endd.append(0)
for year in alldata.index.year.unique():
    subset = alldata[alldata.index.year == year]
    subset.loc[subset['P'] < drempel, 'P'] = 0.
    subset['dry'] = (subset.P == 0.)
    subset['crossing'] = (subset.dry != subset.dry.shift()).cumsum()
    subset['count'] = subset.groupby(['dry', 'crossing']).cumcount(ascending=False) + 1
    subset.loc[subset.dry == False, 'count'] = 0
    subset.loc[subset['dry'],'drynum'] = 1
    subset.loc[~subset['dry'],'drynum'] = 0
    for ind,dat in subset.iterrows():
        if dat.dry:
            duration[n_event]+=1
        else:    
            n_event += 1
            duration.append(0)
            prec_after.append(0)
            years.append(0)
            endd.append(0)
            prec_after[n_event-1] = subset.loc[ind:ind+pd.Timedelta(hours=prec_hours),'P'].sum()
            years[n_event-1] = year
            endd[n_event-1] = ind            

En stop ze in een dataframe

In [ ]:
results = [(s,p,d,y) for s,p,d,y in zip(endd, years, duration, prec_after) if d >= min_duration]
resdf = pd.DataFrame(results)
resdf.columns = ['Enddate', 'Year','Duration','Precipitation']     
resdf.index = resdf['Enddate']
resdf.loc[:,'Duration'] = resdf.Duration / 24.
resdf

In [ ]:
resdf.to_csv(r'tabellen_statistiek/alle_drooogte_gebeurtenissen.csv', index=False, sep=",")

Bovenstaande tabel bevat alle events met een duur langer dan 'min_duration' (nu 3 dagen). De startdatum van het event; het jaar, de duur (in dagen) en de neerslag in de 24-uur eropvolgend.

Plot de duur in uren van alle droogte events. Juni 2023 is best wel extreem....

In [ ]:
ax = plt.subplot()
ax.plot(resdf.index, resdf.Duration, color='blue')
ax.set_ylabel('Duur [dagen]')
plt.savefig(r'figuren_statistiek/droogte_gebeurtenissen.png')

Dit zijn de langste - april/mei 2007 heeft het record: 37 dagen.

In [ ]:
resdf[resdf.Duration > 25]

Is er een relatie tussen de duur en de eropvolgende neerslag? We verwachten het niet.

In [ ]:
ax = plt.subplot()
ax.plot(resdf.Duration, resdf.Precipitation, '.', color='blue')
ax.set_xlabel('Duur droogte [dagen]')
ax.set_ylabel('Neerslag na de droogte [mm]')
ax.set_title('Duur vs neerslag erna')
ax.grid()
plt.savefig(r'figuren_statistiek/droogteduur_vs_eropvolgende_neerslag.png')

Inderdaad.

## Hoeveel van de neerslag raak je kwijt als na elke droogte (> 3 dagen) de eerste 2 mm wordt weggegooid?

Maak een kopie van de resultaten en bewaaar alleen de neerslag na een droogte.

In [ ]:
temp =resdf.copy(deep=True)
temp.drop(['Enddate', 'Duration'], axis=1, inplace=True)

Als de bui groter is dan 2.0 mm wordt de eerste 2.0 mm weggegooid, anders de hele bui

In [ ]:
temp['Gemist'] = [p.Precipitation if p.Precipitation <= 2.0 else 2.0 for _,p in temp.iterrows()]

In [ ]:
temp

Bereken de jaarsommen en vergelijk het met de totale jaarsom (zie boven)

In [ ]:
sommen = temp.groupby("Year").sum()[['Precipitation', 'Gemist']]
sommen['Jaarsom'] = jaarsommen
sommen['Jaarsom_rest'] = sommen['Jaarsom']  - sommen['Gemist'] 
sommen['Perc_gemist'] = (sommen['Gemist'] / sommen['Jaarsom']) * 100.

In [ ]:
sommen

In [ ]:
sommen.to_csv(r'tabellen\gemiste_neerslag_per_jaar.csv', index=False, sep=",")

In [ ]:
fig, ax = plt.subplots(3,1)
sommen['Jaarsom'].plot(ax=ax[0])
sommen['Gemist'].plot(ax=ax[1])
sommen['Perc_gemist'].plot(ax=ax[2])
ax[0].set_ylabel('Jaarsom [mm]')
ax[1].set_ylabel('gemist  [mm]')
ax[2].set_ylabel('% gemist [%]')
ax[0].grid(); ax[1].grid(); ax[2].grid()
plt.savefig(r'figuren_statistiek/gemiste_neersag_per_jaar.png')

## Frequentie van voorkomen van droogtes

Bereken het maximum van elke kolom

In [ ]:
maxima = resdf.groupby('Year').max()

resultaten gebaseerd op de maxima van de droogte

In [ ]:
sorteddf = resdf.copy(deep=True)
sorteddf.index = sorteddf.Enddate
sorteddf = sorteddf.loc[sorteddf[['Year','Duration']].groupby('Year').idxmax()['Duration'].values]

In [ ]:
sorteddf

In [ ]:
sorteddf.to_csv(r'tabellen_statistiek/max_droogteduur_perjaar_met_neerslag.csv', index=False, sep=",")

Bovenstaande tabel laat de langste droogte per jaar zien, met de eropvolgende neerslag.

Nog een check van de relatie: heeft de neerslag volgend op een droogte te maken met de lengte ervan? We plotten ze samen.

In [ ]:
ax = plt.subplot()
ax.plot(sorteddf.Year, sorteddf.Duration, color='blue', label='Max. Duur (d)')
ax.plot(sorteddf.Year, sorteddf.Precipitation, color='orange', label='Neerslag na max. droogte [mm]')
ax.legend()
ax.grid()
ax.set_ylabel('Duur (dagen)/Neerslag [mm]')
ax.set_title('Jaarmaximum droogte-duur met de eropvolgende neerslag')
plt.savefig(r'figuren_statistiek/tijdreeks_droogtes_opvolgendeneerslag.png')

Bepaal hoe vaak droogtes voorkomen. NB: 2007 en 2023 springen er echt uit!

In [ ]:
ht = [1./((i-0.3)/(len(sorteddf.Duration)+0.4)) for i in range(len(sorteddf.Duration))]
ax = plt.subplot()
ax.plot(ht[1:], sorted(sorteddf.Duration[1:], reverse=True), '.')
ax.semilogx()
ax.grid()
ax.set_xlabel('Herhalingstijd [jaren]')
ax.set_ylabel('Droogteduur [dagen]')
ax.set_title('Herhalingstijd van de jaarmaxima van de langste droogte')
plt.savefig(r'figuren_statistiek/jaarmax_droogteduur.png')

Herhalingstijd van buien die na een droogte van tenminste een paar dagen vallen:

In [ ]:
ht = [1./((i-0.3)/(len(maxima.Precipitation)+0.4)) for i in range(len(maxima.Precipitation))]
ax = plt.subplot()
ax.plot(ht[1:], sorted(maxima.Precipitation[1:], reverse=True), '.')
ax.semilogx()
ax.grid()
ax.set_xlabel('Herhalingstijd [jaren]')
ax.set_ylabel('Neerslag na droogte [mm]')
ax.set_title('Herhalingstijd van de jaarmaxima van neerslag na een droogte')
plt.savefig(r'figuren_statistiek/jaarmax_neerslag-na-droogte.png')

In [ ]:
resdf[resdf['Precipitation'] > 50]

## POT statistiek

Peaks over threshold: bepaal alle droogtes met een duur langer dan 2 weken; dus niet alleen het jaarmaximum.

In [ ]:
from pyextremes import get_extremes, get_return_periods
from pyextremes.plotting import plot_extremes

In [ ]:
extremes = get_extremes(resdf.Duration, "POT", threshold=14, r="168H")

In [ ]:
plot_extremes(
    ts=resdf.Duration,
    extremes=extremes,
    extremes_method="POT",
    extremes_type="high",
    threshold=14, # ten minste 2  weken 
    r="168H",
)
plt.savefig(r'figuren_statistiek/peaks_over_threshold.png')

En schat de herhalingstijd ervan.

In [ ]:
return_periods = get_return_periods(
    ts=resdf.Duration,
    extremes=extremes,
    extremes_method="POT",
    extremes_type="high",    
    return_period_size="365.2425D",
    plotting_position="weibull",
)
return_periods.sort_values("return period", ascending=False).head()

Plot de herhalingstijd, maar nu anders (betrouwbaarder?) geschat.

In [ ]:
ax = plt.subplot()
ax.plot(return_periods['return period'], return_periods.Duration,'.')
ax.set_xlabel('Herhalingstijd [jaar]')
ax.set_ylabel('Duur van de droogte [dagen]')
ax.set_title('Droogteduur statistiek op basis van POT')
ax.semilogx()
ax.grid()
plt.savefig(r'figuren_statistiek/herhalingstijd_droogteduur_pot.png')

## Conclusie

- Getallen nog finetunen
- Lange periodes van droogte vooral in het voorjaar, extremen nemen toe (op het oog)
- Periode van ~2 weken komt elk jaar wel eens voor
- Geen relatie met eropvolgende neerslag. Dat kan van alles zijn. 
